In [ ]:
#安裝相關庫
!pip install transformers[torch] accelerate -U
!pip install transformers datasets
!pip install sacrebleu
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
import sacrebleu



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#測試原版gpt2
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 加載原版GPT-2模型和tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
def generate_text(prompt, max_length=1000, num_return_sequences=1, temperature=1.0):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        temperature=temperature,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
    )
    return [tokenizer.decode(seq, skip_special_tokens=True) for seq in output]

# 使用示例
prompt = "Write an adventure story"
generated_text = generate_text(prompt)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Write an adventure story. A good story isn\'t just a quick one. But a story should encompass all of the parts of it and tell that story very clearly.\n\nYou\'ll find that there is good news and bad news about many of them, and that\'s good for you—except you\'ll really have trouble finding what good you want to hear about what. So what should you think about them. In this post, I\'ll give you a few points that will make your next adventure come to life before you\'re done, from beginning to end, thanks to the good advice and the best advice from readers. It won\'t be easy—especially when people are always coming to read your stuff, but there\'s no guarantee that you ever will. That would be the ultimate challenge for the majority of your journey. If you like to be inspired by the stories you read—and maybe just as much your personal life is the story you tell, too—you\'re in luck. Let\'s take this opportunity to get really started.\n\n\nWhat is an Adventure Story\n\n:\nAn Adventure i

In [ ]:
# 使用示例
prompt = "Craft a heartbreaking tale centered around siblings Eliza and Ethan in the town of Willow Creek. When Eliza becomes gravely ill, Ethan strikes a bargain with a shadowy figure to restore her health, oblivious to the dire consequences it will entail."
generated_text = generate_text(prompt)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Craft a heartbreaking tale centered around siblings Eliza and Ethan in the town of Willow Creek. When Eliza becomes gravely ill, Ethan strikes a bargain with a shadowy figure to restore her health, oblivious to the dire consequences it will entail.\n\nThe series takes place in a time in which humankind has become increasingly isolated. The series was made possible because of a combination of innovative filmmaking, unique storytelling and an ability to combine a number of characters into a single whole. Because of our unique relationship with the characters on the show and our ability for the story to be told with empathy and imagination, when it has its share of drama, it becomes necessary to offer a diverse and gripping series that spans the many universes that can exist in these various worlds. "The Dark Side" is a very different series than some of the previous films that will have you in stitches.\n\n\nA New Year, for Everyone on Board\n -\n\n\n, created by: John Z. Hines\n\n']


In [ ]:
# 第一部分：模型訓練

# 從 Hugging Face 加載數據集
dataset = load_dataset("ShehryarAzhar/stories")

# 初始化 tokenizer 和 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 數據預處理（不使用 padding）
def preprocess_function(examples):
    return tokenizer(examples["story"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# 定義一個 collate 函數來處理不同長度的序列
def collate_fn(batch):
    input_ids = [torch.tensor(b["input_ids"]) for b in batch]
    attention_mask = [torch.tensor(b["attention_mask"]) for b in batch]
    labels = input_ids  # 對於語言建模任務，labels 就是 input_ids
    return {
        "input_ids": torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.eos_token_id),
        "attention_mask": torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0),
        "labels": torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)  # -100 是 PyTorch 中用來忽略的 label
    }

# 定義訓練參數
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# 開始訓練
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=collate_fn,
)

trainer.train()

# 保存模型
model.save_pretrained("./trained_model")

Generating train split:   0%|          | 0/1654 [00:00<?, ? examples/s]

Map:   0%|          | 0/1654 [00:00<?, ? examples/s]

Step,Training Loss
500,2.141700
1000,1.656700


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
# 第二部分：故事生成 (保持不變)
def generate_story(prompt, model_path="/content/trained_model"):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=1024, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 使用示例
prompt = "Write an adventure story"
story = generate_story(prompt)
print(story)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write an adventure story with a twist that defied logic and reason.  Enter Detective Sarah Thompson, a seasoned investigator with eyes as sharp as the sharpest blades. As she delved into the world of crime and justice, she encountered a cast of characters unlike any she had ever encountered before. From the ruthless mastermind behind the infamous "Operation Midnight's End," to the enigmatic mastermind who orchestrated the murder of a young woman, Sarah found herself drawn into a world filled with mysteries and intrigue. Each member of the cast seemed to have a motive for wanting justice out of their own lives. Yet, as Thompson delves deeper into Thompson's investigation, the stakes become even higher. With each clue she uncovered, Thompson soon realized that she was not alone. A rival organization, led by a ruthless leader intent on claiming the lives of innocent victims, was also on the loose, intent upon claiming their revenge on Thompson for her investigation. In a heart-stopping cl

In [ ]:
# 使用示例
prompt = "Craft a heartbreaking tale centered around siblings Eliza and Ethan in the town of Willow Creek. When Eliza becomes gravely ill, Ethan strikes a bargain with a shadowy figure to restore her health, oblivious to the dire consequences it will entail."
story = generate_story(prompt)
print(story)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Craft a heartbreaking tale centered around siblings Eliza and Ethan in the town of Willow Creek. When Eliza becomes gravely ill, Ethan strikes a bargain with a shadowy figure to restore her health, oblivious to the dire consequences it will entail.  As Elizabetta embarks on a journey through the wilderness, she faces challenges at every turn—treacherous terrain, treacherous rivers, and the ever-present danger of dehydration. But with each obstacle she overcome, her determination grows stronger, fueled by the promise of redemption and redemption. As she navigates through treacherous terrain and encounters treacherous traps, Elisa discovers signs of her brother's true nature—a thirst for vengeance, a determination to protect her family, an unwavering determination that unites her with the spirits of the past. Yet, as she grows older, the stakes become even higher. With danger lurking around every corner, it's up to Elisab to unravel the mystery of Elistan's tragic demise and save her sib

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

# 評估函數
def calculate_lr_n(story, n=3):
    sentences = sent_tokenize(story)
    total_lr = 0
    for sentence in sentences:
        words = sentence.split()
        lr = len(set(zip(*[words[i:] for i in range(n)]))) / (len(words) - n + 1)
        total_lr += lr
    return total_lr / len(sentences)

def calculate_bleu(generated_story, reference_stories):
    bleu_score = sacrebleu.corpus_bleu([generated_story], [[ref] for ref in reference_stories])
    return bleu_score.score

def evaluate_model(model_path="./trained_model"):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    model.eval()

    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id

    prompts = ["tell a fantasy story", "write a mystery novel", "compose a science fiction tale"]
    generated_stories = [generate_story(prompt, model_path) for prompt in prompts]

    lr_scores = [calculate_lr_n(story) for story in generated_stories]
    avg_lr = sum(lr_scores) / len(lr_scores)
    print(f"Average LR-3 score: {avg_lr}")

    reference_stories = dataset["train"]["story"][:len(prompts)]
    bleu_scores = [calculate_bleu(gen, reference_stories) for gen in generated_stories]
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"Average BLEU score: {avg_bleu}")

# 評估模型
evaluate_model()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Average LR-3 score: 1.0
Average BLEU score: 4.09197992647096
